In [1]:
import numpy as np
import glob

def get_data(files):
    X = np.loadtxt(files[0], dtype=np.int64, delimiter='\t')
    X = X.T
    Y = len(X)*[0]

    for index in range(len(files)-1):
        temp_matrix = np.loadtxt(files[index+1], dtype=np.int64, delimiter='\t')
        temp_matrix = temp_matrix.T

        X = np.concatenate((X, temp_matrix))
        Y = np.concatenate((Y, len(temp_matrix)*[index+1]))

    return X, Y

files = glob.glob('dataset/*.txt')
X, Y = get_data(files)

In [6]:
from sklearn.feature_selection import SelectKBest, chi2

features_rank = SelectKBest(chi2, k=10)
features_rank.fit(X, Y)
selected = features_rank.get_support(True)

for feature in selected:
    print(f"Selected feature: {feature+1} "
          f"with score: {features_rank.scores_[feature]}")



Selected feature: 3 with score: 340.518254073794
Selected feature: 7 with score: 978.5777465914008
Selected feature: 36 with score: 1980.2345300860445
Selected feature: 50 with score: 200.25588898686226
Selected feature: 52 with score: 188.224103615246
Selected feature: 54 with score: 223.46820976892678
Selected feature: 55 with score: 176.9996878058843
Selected feature: 56 with score: 193.3976609689594
Selected feature: 57 with score: 151.67195675769256
Selected feature: 58 with score: 159.89154624936933
